# 4. Games Exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Source: Anton Kozyriev. (2023). Game Recommendations on Steam [Data set]. Kaggle. https://doi.org/10.34740/KAGGLE/DS/2871694

Privacy Statement: The dataset does not contain any personal information about users on a Steam Platform. A preprocessing pipeline anonymized all user IDs. All collected data is accessible to a member of the general public.

## Games

The game dataset contains these columns
- *game_id*: the id of a game
- *title*: the title of a game
- *date_release*: release date
- *win*: if a game is supported on Windows
- *mac*: if a game is supported on Mac
- *linux*: if a game is supported on Linux
- *rating*
- *positive_ratio*
- *user_reviews*
- *price_final*
- *price_original*
- *discount*
- *steam_dec*

In [2]:
games = pd.read_csv("resources/games.csv")
games.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [3]:
print("There are", games['app_id'].nunique(), "distinct games")

There are 50872 distinct games


In [4]:
games[games["app_id"]==10]

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
47375,10,Counter-Strike,2000-11-01,True,True,True,Overwhelmingly Positive,97,136118,9.99,9.99,0.0,True


column: user_reviews. There are fewer reviews in the recommendation dataset compared to the games dataset

## Games Metadata

 There is an additional dataset with games metadata containing description and tags (e.g. Action, Adventure).

In [5]:
games_metadata = pd.read_json("resources/games_metadata.json", lines=True)
games_metadata.head()

,app_id,description,tags
0,13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."
1,22364,,[Action]
2,113020,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
3,226560,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P..."
4,249050,Dungeon of the Endless is a Rogue-Like Dungeon...,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."


In [6]:
games_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50872 entries, 0 to 50871
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   app_id       50872 non-null  int64 
 1   description  50872 non-null  object
 2   tags         50872 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [7]:
print("There are", games_metadata['app_id'].nunique(), "distinct games")

There are 50872 distinct games


Checking if the data frames games and games_metadata have the same games

In [8]:
games_joined = pd.merge(games, games_metadata, left_on="app_id", right_on="app_id", how="outer", indicator=True)
games_joined.groupby("_merge").count()["app_id"]
# games_joined.head(10)

_merge
left_only         0
right_only        0
both          50872
Name: app_id, dtype: int64

Yes, the two data frames can be safely joined.

Now, let's crate a vocabulary of tags that can be used as features in the recommendation systems

In [9]:
def create_tags_vocabulary(games_metadata):
    """
    Creates a vocabulary of games tags

    Args:
        games_metadata::pandas series -  each row contains a list of tags
    Returns:
        tags_vocabulary::vocabulary - keys: tags, values: number of occurrence
    """
    tags_vocabulary = dict()
    for row in games_metadata:
        for tag in row:
            if tag not in tags_vocabulary.keys():
                tags_vocabulary[tag] = 1
            else:
                tags_vocabulary[tag] += 1
    return tags_vocabulary


tags_voc = create_tags_vocabulary(games_metadata["tags"])

print("There are", len(tags_voc), "tags")
print("The top 10 tags with their respective count are:", 
sorted(tags_voc.items(), key=lambda item: -item[1])[:5])

There are 441 tags
The top 10 tags with their respective count are: [('Indie', 27957), ('Singleplayer', 22566), ('Action', 21897), ('Adventure', 20183), ('Casual', 17844)]


Creating a data frame having games to the rows and tags to columns. This can be used later for different types for the different recommendation systems (i.e. content-based filtering, collaborative filtering)

In [10]:
def get_matrix_games_tags(df):
    """
    Args:
        df::dataframe, e.g. games_metadata
    Returns:
        games_tags:: dataframe, games-tag matrix

    This creates a games-tag matrix similar to 
    app_id | Action | RPG | Adventure |
    -------|--------|-----|-----------|
    1545830|        |  1  |           |
    9879765|    1   |     |           |
    
    """
    # creating a new column passing 1. Useful for the final sparse matrix
    df["has_tag"] = 1
    # exploding the tags array into rows
    df = df[["app_id","tags","has_tag"]].explode("tags")
    games_tags = df.pivot_table(values="has_tag", index="app_id", columns="tags")
    games_tags = games_tags.fillna(0)
    games_tags = games_tags.astype("int8")
    return games_tags

matrix_games_tags = get_matrix_games_tags(games_metadata)
matrix_games_tags.head()

tags,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
app_id,,,,,,,,,,,,,,,,,,,,,
10,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
40,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
matrix_games_tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49628 entries, 10 to 2515460
Columns: 441 entries, 1980s to eSports
dtypes: int8(441)
memory usage: 21.3 MB
